This notebook has been used on [Kaggle](https://www.kaggle.com/).

In [ ]:
# Clone my repository
!git clone https://github.com/danielpetrov18/Evaluation-Approaches-for-Retrieval-Augmented-Generation-RAG-.git

In [1]:
# Switch into the ragas folder
%cd Evaluation-Approaches-for-Retrieval-Augmented-Generation-RAG-/evaluation/ragas

/kaggle/working/Evaluation-Approaches-for-Retrieval-Augmented-Generation-RAG-/evaluation/ragas


In [ ]:
# Install dependencies
!pip3 install -U ragas==0.2.15 rapidfuzz==3.13.0 langchain-ollama==0.3.2 python-dotenv==1.1.0

In [ ]:
# Download and install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [2]:
import os
import subprocess

# Start Ollama server in the background
ollama_process = subprocess.Popen(
    ["ollama", "serve"], 
)

2025/05/13 12:55:43 routes.go:1233: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES: http_proxy: https_proxy:

In [ ]:
from dotenv import load_dotenv

# Download models
!ollama pull mxbai-embed-large && llama3.1:8b-instruct-q4_1

# Enhance llama3.1:8b-instruct-q4_1 with greater context window size
load_dotenv("../../env/rag.env")

! echo -e "FROM $CHAT_MODEL\nPARAMETER num_ctx $LLM_CONTEXT_WINDOW_TOKENS" > Modelfile
! ollama create "$CHAT_MODEL" -f Modelfile

In [3]:
# Retrieve sensitive data
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

ragas_app_token = user_secrets.get_secret("RAGAS_APP_TOKEN")

os.environ['RAGAS_APP_TOKEN'] = ragas_app_token 

In [4]:
import json
from typing import List, Dict
from ragas import EvaluationDataset, SingleTurnSample

# Load the dataset corresponding to the experiment you want to test

# The are located under `./datasets`
filepath: str = input("Please specify which dataset to evaluate (only the file name): ")

goldens: List[Dict] = []
try:
    with open(file=f"./datasets/{filepath}.jsonl", mode="r", encoding="utf-8") as file:
        for line in file:
            if line.strip():  # Skip empty lines
                goldens.append(json.loads(line))

    samples: List[SingleTurnSample] = []
    for golden in goldens:
        single_turn_sample = SingleTurnSample(**golden)
        samples.append(single_turn_sample)
        
    evaluation_dataset = EvaluationDataset(samples)
except FileNotFoundError:
    print(f"File: `./datasets/{filepath}.jsonl` containing goldens not found!")
except json.JSONDecodeError as e:
    print(f"Error parsing JSONL file: {e}")

Please specify which dataset to evaluate (only the file name):  test_id_1-dataset


In [5]:
import os
from dotenv import load_dotenv
from langchain_ollama import ChatOllama, OllamaEmbeddings

from ragas import RunConfig, DiskCacheBackend
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Instantiating required objects

run_config = RunConfig(
    timeout=86400,    # 24 hours on waiting for a single operation
    max_retries=20,   # Max retries before giving up
    max_wait=600,     # Max wait between retries
    max_workers=4,    # Concurrent requests
    log_tenacity=True # Print retry attempts
)

# This stores data generation and evaluation results locally on disk
# When using it for the first time, it will create a .cache folder
# When using it again, it will read from that folder and finish almost instantly
cacher = DiskCacheBackend(cache_dir=f".cache-{filepath}")

load_dotenv("../../env/rag.env")

ragas_llm = LangchainLLMWrapper(
    langchain_llm=ChatOllama(
        model=os.getenv("CHAT_MODEL"),
        base_url="http://localhost:11434",
        temperature=float(os.getenv("TEMPERATURE")),
        num_ctx=int(os.getenv("LLM_CONTEXT_WINDOW_TOKENS")),
        format="json"
    ),
    run_config=run_config,
    cache=cacher
)

ragas_embeddings = LangchainEmbeddingsWrapper(
    embeddings=OllamaEmbeddings(
        model=os.getenv("EMBEDDING_MODEL"),
        base_url="http://localhost:11434"
    ),
    run_config=run_config,
    cache=cacher
)

In [6]:
from ragas.metrics import LLMContextPrecisionWithReference
from prompts.metrics.custom_context_precision_prompt import MyContextPrecisionPrompt

context_precision = LLMContextPrecisionWithReference(
    name="context_precision",
    context_precision_prompt=MyContextPrecisionPrompt(),
    max_retries=20
)

In [7]:
from ragas.metrics import LLMContextRecall   
from prompts.metrics.custom_context_recall_prompt import MyContextRecallPrompt

context_recall = LLMContextRecall(
    name="context_recall",
    context_recall_prompt=MyContextRecallPrompt(),
    max_retries=20
)

In [8]:
from ragas.metrics import ContextEntityRecall
from prompts.metrics.custom_context_entities_recall_prompt import MyContextEntitiesRecallPrompt

context_entity_recall = ContextEntityRecall(
    context_entity_recall_prompt=MyContextEntitiesRecallPrompt(),
    max_retries=20
)

In [9]:
from ragas.metrics import NoiseSensitivity
from prompts.metrics.faithfulness.custom_nli_generator_prompt import MyNLIStatementPrompt
from prompts.metrics.faithfulness.custom_statement_generator_prompt import MyStatementGeneratorPrompt

noise_sensitivity = NoiseSensitivity(
    nli_statements_prompt=MyNLIStatementPrompt(),
    statement_generator_prompt=MyStatementGeneratorPrompt(),
    max_retries=20
)

In [10]:
from ragas.metrics import ResponseRelevancy
from prompts.metrics.custom_response_relevance_prompt import MyResponseRelevancePrompt

response_relevancy = ResponseRelevancy(
    question_generation=MyResponseRelevancePrompt()
)

In [11]:
from ragas.metrics import Faithfulness
from prompts.metrics.faithfulness.custom_nli_generator_prompt import MyNLIStatementPrompt
from prompts.metrics.faithfulness.custom_statement_generator_prompt import MyStatementGeneratorPrompt

faithfulness = Faithfulness(
    nli_statements_prompt=MyNLIStatementPrompt(),
    statement_generator_prompt=MyStatementGeneratorPrompt(),
    max_retries=20,
)

In [12]:
from ragas.metrics import FactualCorrectness
from prompts.metrics.faithfulness.custom_nli_generator_prompt import MyNLIStatementPrompt

factual_correctness = FactualCorrectness(
    nli_prompt=MyNLIStatementPrompt()
)

In [13]:
from ragas.metrics import SemanticSimilarity

semantic_similarity = SemanticSimilarity(
    threshold=0.7, # Default is 0.5 = 50%
)

In [14]:
from ragas.evaluation import evaluate, EvaluationResult

results: EvaluationResult = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        context_precision,           # Metric 1
        context_recall,              # Metric 2
        context_entity_recall,       # Metric 3
        noise_sensitivity,           # Metric 4
        response_relevancy,          # Metric 5
        faithfulness,                # Metric 6
        factual_correctness,         # Metric 7
        semantic_similarity          # Metric 8
    ],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    experiment_name=f"{filepath}-evaluation",
    run_config=run_config,
    show_progress=True
)

Evaluating:   0%|          | 0/408 [00:00<?, ?it/s]

time=2025-05-13T12:56:20.553Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-13T12:56:20.557Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-13T12:56:20.575Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-13T12:56:20.577Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-13T12:56:20.832Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-13T12:56:20.854Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32
time=2025-05-13T12:56:20.855Z level=WARN source=ggml.go:152 msg="key not found" key=llama.vision.block_count default=0
time=2025-05-13T12:56:20.855Z level=WARN source=ggml.go:152 msg="key not found" key=llama.attention.key_length default=128
time=2025-05-13T12:56:20.855Z level=WARN source=ggml.go:152 msg="key not found" 

[GIN] 2025/05/13 - 12:56:34 | 200 | 13.913590431s |       127.0.0.1 | POST     "/api/chat"


time=2025-05-13T12:56:34.472Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/13 - 12:56:37 | 200 | 16.589974523s |       127.0.0.1 | POST     "/api/chat"


...

[GIN] 2025/05/13 - 15:20:24 | 200 | 47.004749127s |       127.0.0.1 | POST     "/api/chat"


time=2025-05-13T15:20:24.117Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/13 - 15:20:32 | 200 |  8.823515808s |       127.0.0.1 | POST     "/api/chat"


time=2025-05-13T15:20:32.947Z level=WARN source=ggml.go:152 msg="key not found" key=general.alignment default=32


[GIN] 2025/05/13 - 15:20:42 | 200 |  9.193628353s |       127.0.0.1 | POST     "/api/chat"


In [15]:
import pandas as pd

# Save results locally (optional)
result_df: pd.DataFrame = results.to_pandas()
result_df.to_csv(f'./{filepath}-eval-results.csv', index=False)

# Display metric scores
results

{'context_precision': 0.2908, 'context_recall': 0.8472, 'context_entity_recall': 0.2305, 'noise_sensitivity(mode=relevant)': 0.2946, 'answer_relevancy': 0.8493, 'faithfulness': 0.8770, 'factual_correctness(mode=f1)': 0.5404, 'semantic_similarity': 1.0000}

In [16]:
results.upload()

Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/6c779059-92f2-4f98-866a-7c87f2d787d0


'https://app.ragas.io/dashboard/alignment/evaluation/6c779059-92f2-4f98-866a-7c87f2d787d0'

In [17]:
! ls

dataset-goldens.jsonl  fill_dataset.py	Modelfile
datasets	       fill_dataset.sh	prompts
evaluate.ipynb	       generate.ipynb	test_id_1-dataset-eval-results.csv
